In [ ]:
import torch

In [ ]:
def activation(x: torch.Tensor, function: str) -> torch.Tensor:
    if function == 'relu':
        return torch.where(x > 0, x, torch.zeros_like(x))
    elif function == 'sigmoid':
        return 1 / (1 + torch.exp(-x))
    elif function == 'tanh':
        return torch.tanh(x)
    elif function == 'linear':
        return x
    elif function == 'softmax':
        exp_x = torch.exp(x - torch.max(x, dim=0, keepdim=True)[0])  
        return exp_x / torch.sum(exp_x, dim=0, keepdim=True)
    else:
        raise ValueError(f"Activation function {function} not supported")

In [ ]:
def act_der(x: torch.Tensor, function: str) -> torch.Tensor:
    if function == 'relu':
        return (x > 0).float()
    elif function == 'sigmoid':
        sig = activation(x, 'sigmoid')
        return sig * (1 - sig)
    elif function == 'tanh':
        return 1 - activation(x, 'tanh').pow(2)
    elif function == 'linear':
        return torch.ones_like(x)
    elif function == 'softmax':
        # For softmax, we don't calculate the derivative directly in backprop
        # Instead, we use the loss derivative which already accounts for softmax derivative
        return torch.ones_like(x)
    else:
        raise ValueError(f"Activation function {function} not supported")

In [ ]:
def loss(gt: torch.Tensor, pred: torch.Tensor, function: str) -> torch.Tensor:
    if gt.device != pred.device:
        gt = gt.to(pred.device)
        
    if function == 'mse':
        return torch.mean((pred - gt).pow(2))
    elif function == 'bce':
        return -torch.mean(gt * torch.log(pred + 1e-7) + (1 - gt) * torch.log(1 - pred + 1e-7))
    elif function == 'ce':  # Cross-entropy loss for multi-class classification
        return -torch.mean(torch.sum(gt * torch.log(pred + 1e-7), dim=0))

In [ ]:
def loss_der(gt: torch.Tensor, pred: torch.Tensor, function: str) -> torch.Tensor:
    if gt.device != pred.device:
        gt = gt.to(pred.device)
        
    if function == 'mse':
        return 2 * (pred - gt) / gt.size(1)
    elif function == 'bce':
        return -gt / (pred + 1e-7) + (1 - gt) / (1 - pred + 1e-7)
    elif function == 'ce':
        # For cross-entropy loss with softmax, the derivative simplifies to (pred - gt)
        # when each sample has only one correct class
        return (pred - gt) / gt.size(1)

In [ ]:
class NeuralNetwork:
    def __init__(self, device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.layers = []
        self.z_list = []
        self.a_list = []
        self.device = device

    def add_layer(self, input_size: int, output_size: int, activation: str):
        # Initialize weights using He initialization for ReLU or Xavier for sigmoid/tanh
        if activation == 'relu':
            w = torch.randn(output_size, input_size, device=self.device) * torch.sqrt(torch.tensor(2.0 / input_size, device=self.device))
        else:
            w = torch.randn(output_size, input_size, device=self.device) * torch.sqrt(torch.tensor(1.0 / input_size, device=self.device))
            
        b = torch.zeros(output_size, 1, device=self.device)
        self.layers.append([w, b, activation])
        self.z_list.append(None)
        self.a_list.append(None)

    def forwardprop(self, x: torch.Tensor) -> torch.Tensor:
        current_input = x.to(self.device)  
        for i in range(len(self.layers)):
            w, b, activation_func = self.layers[i]
            z = torch.mm(w, current_input) + b
            self.z_list[i] = z
            self.a_list[i] = activation(z, activation_func)
            current_input = self.a_list[i]
        return self.a_list[-1]

    def backprop(self, x: torch.Tensor, y: torch.Tensor, loss_function: str, learning_rate: float = 0.01):
        m = x.size(1)  
        self.forwardprop(x)
        
        dz = loss_der(y, self.a_list[-1], loss_function)
        for i in range(len(self.layers)-1, -1, -1):
            w, b, activation_func = self.layers[i]
            a_prev = self.a_list[i-1] if i > 0 else x
            
            # If the last layer uses softmax with cross-entropy, dz is already computed correctly
            if i == len(self.layers)-1 and activation_func == 'softmax' and loss_function == 'ce':
                # Skip multiplying by activation derivative - already accounted for in cross-entropy derivative
                pass
            else:
                dz = dz * act_der(self.z_list[i], activation_func)
            
            dw = (1/m) * torch.mm(dz, a_prev.T)
            db = (1/m) * torch.sum(dz, dim=1, keepdim=True)
            
            self.layers[i][0] = w - learning_rate * dw
            self.layers[i][1] = b - learning_rate * db
            
            if i > 0:
                dz = torch.mm(w.T, dz)

    def train(self, X: torch.Tensor, y: torch.Tensor, epochs: int = 100, 
             batch_size: int = 32, learning_rate: float = 0.01, 
             loss_function: str = 'mse', optimizer: str = 'Mini_Batch', verbose: bool = True):
        n_samples = X.size(1)  
        
        X = X.to(self.device) 
        y = y.to(self.device)  
        
        epoch_losses = []

        for epoch in range(epochs):
            total_loss = 0.0
            perm = torch.randperm(n_samples, device=self.device)
            X_shuffled = X[:, perm]
            y_shuffled = y[:, perm]  
            
            if optimizer == 'Batch':
                # Process all samples at once
                pred = self.forwardprop(X_shuffled)
                batch_loss = loss(y_shuffled, pred, loss_function)
                total_loss += batch_loss.item()
                self.backprop(X_shuffled, y_shuffled, loss_function, learning_rate)
            
            elif optimizer == 'SGD':
                # Stochastic Gradient Descent - process one sample at a time
                for i in range(n_samples):
                    X_sample = X_shuffled[:, i:i+1]  # Get a single sample
                    y_sample = y_shuffled[:, i:i+1]  # Get corresponding label
                    
                    pred = self.forwardprop(X_sample)
                    sample_loss = loss(y_sample, pred, loss_function)
                    total_loss += sample_loss.item()
                    
                    self.backprop(X_sample, y_sample, loss_function, learning_rate)
                
                total_loss = total_loss / n_samples  # Average the loss over samples
            
            else:  # Mini_Batch (default)
                n_batches = (n_samples + batch_size - 1) // batch_size
                
                for batch in range(n_batches):
                    start_idx = batch * batch_size
                    end_idx = min((batch + 1) * batch_size, n_samples)
                    
                    X_batch = X_shuffled[:, start_idx:end_idx]
                    y_batch = y_shuffled[:, start_idx:end_idx]
                    
                    pred = self.forwardprop(X_batch)
                    batch_loss = loss(y_batch, pred, loss_function)
                    total_loss += batch_loss.item()
                    
                    self.backprop(X_batch, y_batch, loss_function, learning_rate)
                
                total_loss = total_loss / n_batches  # Average loss over batches
            
            epoch_losses.append(total_loss)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")
                
        return epoch_losses  # Return the loss history

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        return self.forwardprop(X)

    def evaluate(self, X: torch.Tensor, y: torch.Tensor, loss_function: str = 'mse') -> torch.Tensor:
        X = X.to(self.device)
        y = y.to(self.device)
        pred = self.forwardprop(X)
        return loss(y, pred, loss_function)
    
    def accuracy(self, X: torch.Tensor, y: torch.Tensor) -> float:
        X = X.to(self.device)
        y = y.to(self.device)
        
        pred = self.forwardprop(X)
        pred_classes = torch.argmax(pred, dim=0)
        true_classes = torch.argmax(y, dim=0)
        
        correct = (pred_classes == true_classes).float().sum()
        total = y.size(1)
        
        return (correct / total).item() * 100

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import glob
import re
import pickle

In [ ]:
# Data loading and preparation functions
def load_images(image_dir="/kaggle/input/smai-symbol-data/images/"):
    if os.path.exists("symbols_images.npy"):
        print("Loading cached images...")
        images = np.load("symbols_images.npy", allow_pickle=True)
        if len(images) > 0:
            print(f"Successfully loaded {len(images)} cached images.")
            return images
        else:
            print("Cached images file is empty. Reloading from directory...")
    
    print("Loading images from directory...")
    image_paths = glob.glob(image_dir + "*.png")
    
    if not image_paths:
        raise ValueError(f"No PNG images found in directory: {image_dir}")
    
    print(f"Found {len(image_paths)} image files.")
    
    def extract_number(path):
        match = re.search(r"(\d+)\.png$", path)
        return int(match.group(1)) if match else float('inf')
    
    image_paths = sorted(image_paths, key=extract_number)
    
    import cv2
    images = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Failed to load image: {path}")
            continue
        images.append(img)
    
    if not images:
        raise ValueError("No images were successfully loaded!")
    
    images = np.array(images, dtype=np.uint8)
    print(f"Successfully loaded {len(images)} images.")
    
    # Save images for future use
    np.save("symbols_images.npy", images)
    print(f"Saved {len(images)} images to symbols_images.npy.")
    
    return images

In [ ]:
def load_folds(base_dir="/kaggle/input/smai-symbol-data/classification-task"):
    data = {}
    pattern = re.compile(r"(\d+)\.png$")
    
    for i in range(1, 11):
        fold = f"fold-{i}"
        fold_dir = os.path.join(base_dir, fold)
        train_csv = os.path.join(fold_dir, "train.csv")
        test_csv = os.path.join(fold_dir, "test.csv")
        
        train_df = pd.read_csv(train_csv)
        test_df = pd.read_csv(test_csv)
        
        train_df.drop(columns=["user_id"], errors="ignore", inplace=True)
        test_df.drop(columns=["user_id"], errors="ignore", inplace=True)
        
        train_df["index"] = train_df["path"].apply(lambda x: int(pattern.search(x).group(1)))
        test_df["index"] = test_df["path"].apply(lambda x: int(pattern.search(x).group(1)))
        
        # Store processed data
        data[fold] = {
            "train": train_df,
            "test": test_df
        }

    return data

In [ ]:
def load_symbols_mapping(symbol_path="/kaggle/input/smai-symbols/symbols.csv"):
    """Load symbol ID to LaTeX mapping."""
    symbols_df = pd.read_csv(symbol_path)
    unique_symbol_ids = sorted(symbols_df["symbol_id"].unique())
    symbol_id_to_index = {symbol_id: i for i, symbol_id in enumerate(unique_symbol_ids)}
    index_to_symbol_id = {i: symbol_id for symbol_id, i in symbol_id_to_index.items()}
    return symbols_df, symbol_id_to_index, index_to_symbol_id

In [ ]:
def prepare_data(fold_data, images, symbol_id_to_index, num_classes):
    """Prepare data for training and testing."""
    if len(images) == 0:
        raise ValueError("No images available for data preparation!")
    
    X = []
    y = []
    
    for _, row in fold_data.iterrows():
        idx = row["index"]
        if idx >= len(images):
            print(f"Warning: Index {idx} is out of bounds for images array of size {len(images)}")
            continue
            
        X.append(images[idx].flatten())
        y.append(symbol_id_to_index[row["symbol_id"]])
    
    if not X:
        raise ValueError("No valid data points were prepared!")
    
    X = np.array(X).T / 255.0  # Normalize and reshape to (1024, n_samples)
    y = np.array(y)
    
    # One-hot encode the labels
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, np.arange(y.shape[0])] = 1

    X = torch.tensor(X, dtype=torch.float32)
    y_one_hot = torch.tensor(y_one_hot, dtype=torch.float32)
    
    return X, y_one_hot, y  # Return y (non-one-hot) for evaluation

In [ ]:
def train_and_evaluate(fold, images, data, symbol_id_to_index, index_to_symbol_id, 
                       num_classes, hidden_sizes, activation_func, optimizer, 
                       learning_rate, epochs, batch_size):
    """Train and evaluate model for a specific fold and configuration."""
    if len(images) == 0:
        raise ValueError("No images available for training!")
        
    train_data = data[f"fold-{fold}"]["train"]
    test_data = data[f"fold-{fold}"]["test"]
    
    print(f"Processing fold {fold}:")
    print(f"  Training data size: {len(train_data)}")
    print(f"  Testing data size: {len(test_data)}")
    
    X_train, y_train_one_hot, y_train = prepare_data(train_data, images, symbol_id_to_index, num_classes)
    X_test, y_test_one_hot, y_test = prepare_data(test_data, images, symbol_id_to_index, num_classes)
    
    print(f"  Prepared training data shape: {X_train.shape}")
    print(f"  Prepared testing data shape: {X_test.shape}")
    
    # Create and train the model
    model = NeuralNetwork()  # Use GPU if available
    
    # Add layers
    model.add_layer(X_train.shape[0], hidden_sizes[0], activation_func)
    for i in range(1, len(hidden_sizes)):
        model.add_layer(hidden_sizes[i-1], hidden_sizes[i], activation_func)
    model.add_layer(hidden_sizes[-1], num_classes, 'softmax')  # Output layer with softmax
    
    # Train the model
    history = model.train(X_train, y_train_one_hot, epochs=epochs, 
                         batch_size=batch_size, learning_rate=learning_rate, 
                         loss_function='ce', optimizer=optimizer, verbose=True)
    
    # Make predictions
    predictions = model.forwardprop(X_test)
    pred_classes = torch.argmax(predictions, dim=0).cpu().numpy()
    true_classes = y_test
    
    # Convert to symbol IDs for reporting
    predicted_symbol_ids = [index_to_symbol_id[p] for p in pred_classes]
    actual_symbol_ids = [index_to_symbol_id[y] for y in true_classes]
    
    # Calculate accuracy
    accuracy = np.mean(np.array(pred_classes) == np.array(true_classes))
    
    return accuracy, history

In [ ]:
def run_hyperparameter_tuning():
    """Run hyperparameter tuning with 10-fold validation."""
    print("Loading dataset...")
    images = load_images()
    data = load_folds()
    symbols_df, symbol_id_to_index, index_to_symbol_id = load_symbols_mapping()
    num_classes = len(symbol_id_to_index)
    
    print(f"Dataset loaded. Number of classes: {num_classes}")
    
    # Define hyperparameters to tune
    hidden_sizes_options = [
        [512, 256], 
        [1024, 512]
    ]
    activation_options = ['sigmoid', 'tanh', 'relu']
    optimizer_options = ['Batch', 'SGD', 'Mini_Batch']
    learning_rates = [0.01, 0.001]
    batch_sizes = [32, 64]  # Only relevant for Mini_Batch
    epochs = 20
    
    results = {}
    
    # Run experiments
    for hidden_sizes in hidden_sizes_options:
        for activation_func in activation_options:
            for optimizer in optimizer_options:
                for lr in learning_rates:
                    # Skip batch size loop for Batch and SGD optimizers
                    if optimizer == 'Batch':
                        batch_size_values = [0]  # Not used for Batch
                    elif optimizer == 'SGD':
                        batch_size_values = [1]  # Not used for SGD
                    else:  # Mini_Batch
                        batch_size_values = batch_sizes
                    
                    for batch_size in batch_size_values:
                        config_name = f"hs:{hidden_sizes}_act:{activation_func}_opt:{optimizer}_lr:{lr}"
                        if optimizer == 'Mini_Batch':
                            config_name += f"_bs:{batch_size}"
                        
                        print(f"\nRunning configuration: {config_name}")
                        
                        accuracies = []
                        histories = []
                        
                        # Run for all 10 folds or a subset for testing
                        for fold in range(1, 11):
                            print(f"  Processing fold {fold}...")
                            accuracy, history = train_and_evaluate(
                                fold, images, data, symbol_id_to_index, index_to_symbol_id,
                                num_classes, hidden_sizes, activation_func, optimizer,
                                lr, epochs, batch_size if optimizer == 'Mini_Batch' else None
                            )
                            
                            accuracies.append(accuracy)
                            histories.append(history)
                            
                            print(f"  Fold {fold} accuracy: {accuracy:.4f}")
                        
                        results[config_name] = {
                            'accuracies': accuracies,
                            'mean_accuracy': np.mean(accuracies),
                            'std_accuracy': np.std(accuracies),
                            'histories': histories,
                            'config': {
                                'hidden_sizes': hidden_sizes,
                                'activation_func': activation_func,
                                'optimizer': optimizer,
                                'learning_rate': lr,
                                'batch_size': batch_size if optimizer == 'Mini_Batch' else None
                            }
                        }
                        
                        print(f"Configuration {config_name}:")
                        print(f"  Mean accuracy: {results[config_name]['mean_accuracy']:.4f}")
                        print(f"  Std accuracy: {results[config_name]['std_accuracy']:.4f}")
    
    with open('mlp_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    return results

In [ ]:
def visualize_results(results):
    """Visualize the results of hyperparameter tuning."""

    sorted_configs = sorted(
        results.items(), 
        key=lambda x: x[1]['mean_accuracy'], 
        reverse=True
    )
    
    plt.figure(figsize=(14, 8))
    
    config_names = [config[:20] + '...' if len(config) > 20 else config for config, _ in sorted_configs[:10]]
    means = [res['mean_accuracy'] for _, res in sorted_configs[:10]]
    stds = [res['std_accuracy'] for _, res in sorted_configs[:10]]
    
    plt.bar(config_names, means, yerr=stds, capsize=5)
    plt.ylabel('Mean Accuracy')
    plt.xlabel('Configuration')
    plt.title('Top 10 Configurations by Mean Accuracy')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    plt.savefig('top_configurations.png')
    
    plot_by_category(results, 'activation_func', 'Activation Function')
    
    plot_by_category(results, 'optimizer', 'Optimizer')
    
    plot_training_curves(sorted_configs[:3])

In [ ]:
def plot_by_category(results, category, title):
    categories = {}
    
    for config_name, result in results.items():
        cat_value = result['config'][category]
        if cat_value not in categories:
            categories[cat_value] = []
        categories[cat_value].append(result['mean_accuracy'])
    
    cat_means = {cat: np.mean(values) for cat, values in categories.items()}
    cat_stds = {cat: np.std(values) for cat, values in categories.items()}
    
    # Sort by mean
    sorted_cats = sorted(cat_means.items(), key=lambda x: x[1], reverse=True)
    
    # Plot
    plt.figure(figsize=(10, 6))
    cat_names = [cat for cat, _ in sorted_cats]
    means = [mean for _, mean in sorted_cats]
    stds = [cat_stds[cat] for cat, _ in sorted_cats]
    
    plt.bar(cat_names, means, yerr=stds, capsize=5)
    plt.ylabel('Mean Accuracy')
    plt.xlabel(title)
    plt.title(f'Performance by {title}')
    plt.ylim(0.5, 1.0)
    plt.tight_layout()
    plt.show()
    plt.savefig(f'performance_by_{category}.png')

In [ ]:
def plot_training_curves(top_configs):
    """Plot training curves for top configurations."""
    plt.figure(figsize=(12, 6))
    
    for idx, (config_name, result) in enumerate(top_configs):
        # Average the training curves across folds
        avg_history = np.mean(result['histories'], axis=0)
        epochs = np.arange(1, len(avg_history) + 1)
        
        plt.plot(epochs, avg_history, label=f"Config {idx+1}: {config_name[:30]}...")
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Curves for Top Configurations')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('training_curves.png')

In [ ]:
def analyze_results(results):
    """Analyze and report on the results."""
    # Sort configurations by mean accuracy
    sorted_configs = sorted(
        results.items(), 
        key=lambda x: x[1]['mean_accuracy'], 
        reverse=True
    )
    
    # Display top configurations
    print("\nTop 5 Configurations:")
    for i, (config_name, result) in enumerate(sorted_configs[:5]):
        print(f"{i+1}. {config_name}")
        print(f"   Mean Accuracy: {result['mean_accuracy']:.4f}")
        print(f"   Std Accuracy: {result['std_accuracy']:.4f}")
        print(f"   Config: {result['config']}")
        print()
    
    # Calculate average performance by activation function
    activations = {}
    for config_name, result in results.items():
        act = result['config']['activation_func']
        if act not in activations:
            activations[act] = []
        activations[act].append(result['mean_accuracy'])
    
    print("\nPerformance by Activation Function:")
    for act, accuracies in activations.items():
        print(f"{act}: Mean={np.mean(accuracies):.4f}, Std={np.std(accuracies):.4f}")
    
    # Calculate average performance by optimizer
    optimizers = {}
    for config_name, result in results.items():
        opt = result['config']['optimizer']
        if opt not in optimizers:
            optimizers[opt] = []
        optimizers[opt].append(result['mean_accuracy'])
    
    print("\nPerformance by Optimizer:")
    for opt, accuracies in optimizers.items():
        print(f"{opt}: Mean={np.mean(accuracies):.4f}, Std={np.std(accuracies):.4f}")
    
    


In [ ]:
if __name__ == "__main__":
    # Run the complete pipeline
    results = run_hyperparameter_tuning()
    visualize_results(results)
    analyze_results(results) 

### Answers to Report Questions

#### 1. Mean and Standard Deviation Interpretation
- The **mean accuracy** reflects the overall performance of a configuration across all folds.
- A **higher mean** indicates better general performance.
- The **standard deviation** shows how consistent the model performs across different folds.
- A **lower standard deviation** indicates more consistent performance across different data splits.

#### 2. Impact of High vs. Low Standard Deviation
- A **high standard deviation** suggests the model's performance varies significantly depending on the data split,
  - which indicates that the model might be sensitive to the specific examples in the training/test sets.
  - This reduces confidence in the model's ability to generalize to unseen data.
- A **low standard deviation** indicates consistent performance regardless of the data split,
  - which increases confidence in the model's generalization capabilities.

#### 3. Choosing Between Configurations with Different Means and Standard Deviations
- When choosing between a configuration with **slightly higher mean accuracy but significantly higher**
  **standard deviation** versus one with **marginally lower mean accuracy but much lower standard deviation,**
  - The configuration with the **lower standard deviation** is generally preferable.
  - The small sacrifice in average performance is outweighed by the increased reliability and consistency,
    - especially important in real-world applications where consistent performance is critical.
    - The more stable model is **less likely to fail catastrophically** on certain data distributions.
